In [56]:
# Dependencies 
import pandas as pd
import os
from datetime import datetime
from bs4 import BeautifulSoup
import glob as glob

In [57]:
# Import alternate championhsip points tables
points_table_formula1 = pd.read_csv('points_systems_csv/2024_formula1_race_points_table.csv')
points_table_imsa = pd.read_csv('points_systems_csv/2024_imsa_race_points_table.csv')
points_table_imsa_qual = pd.read_csv('points_systems_csv/2024_imsa_qualifying_points_table.csv')


In [58]:
all_files_path = 'race_results_html/*.htm'
race_results_files = []
race_results_files = glob.glob(all_files_path)

season_races_df = pd.DataFrame()

# Initialize a list to contain the race results 
race_result_data = []
i = 0

for race_result_file in race_results_files:
# Get City / Race Name
    i += 1
    j = 0
    city = race_result_file.split('.')[0]
    city = city[18:]
# Read html file and store as html_content
    with open(race_result_file, 'r') as file:
        html_content = file.read()
    # Parse race result table html
    soup = BeautifulSoup(html_content, 'html.parser')
    race_table_html = soup.find(id = 'race-season')

# Loop through table html and parse to list
    for row in race_table_html.find_all('tr'):
        j += 1
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
        if j == 1 and i == 1:
            row_data_with_city = row_data + ['City']
            race_result_data.append(row_data_with_city)
        elif j != 1:
            row_data_with_city = row_data + [city]
            race_result_data.append(row_data_with_city)
        else:
            pass

    race_result_df = pd.DataFrame(race_result_data[1:], columns=race_result_data[0])
    race_result_df['Rank'] = race_result_df['Rank'].astype('int')
    race_result_df['Starts'] = race_result_df['Starts'].astype('int')
    race_result_df['Laps'] = race_result_df['Laps'].astype('int')
    race_result_df['Laps Led'] = race_result_df['Laps'].astype('int')
    race_result_df['Points'] = race_result_df['Points'].astype('int')
    race_result_df['Pit stop'] = race_result_df['Pit stop'].astype('int64')

    race_result_df = pd.merge(race_result_df, points_table_formula1, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa_qual, on='Starts', how='inner')
    race_result_df['Points (IMSA Scoring)'] = race_result_df['Points (IMSA Scoring)']+race_result_df['Points (IMSA Scoring) - Qualifying']
    race_result_df = race_result_df.drop(columns=['Points (IMSA Scoring) - Qualifying'])

    

race_result_df.sample(46)

,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Points (F1 Scoring),Points (IMSA Scoring)
44,18,Sting Ray Robb,41,25,85,01:43:08.7314,85,Running,12,97.307,2,longbeach,0,136
13,14,Rinus VeeKay,21,18,100,02:06:37.3806,100,Running,16,77.948,3,detroit,0,183
342,12,Rinus VeeKay,21,10,205,01:43:32.1299,205,Running,18,158.004,4,nashville,0,211
439,1,Alex Palou,10,1,95,02:04:09.8545,95,Running,54,102.740,3,lagunaseca,25,385
323,20,Jack Harvey,18,26,249,02:13:44.4603,249,Contact,10,139.636,4,gateway,0,115
315,12,Santino Ferrucci,14,17,258,02:22:35.5855,258,Running,18,135.701,5,gateway,0,204
23,24,Kyffin Simpson,4,27,96,02:06:56.8918,96,Running,6,74.638,6,detroit,0,74
196,7,Christian Lundgaard,45,9,80,01:33:55.1905,80,Running,26,115.401,2,midohio,6,262
143,9,Santino Ferrucci,14,14,100,01:51:52.6391,100,Running,22,96.534,2,stpetersburg,2,237
235,19,Romain Grosjean,77,26,200,02:58:59.2391,200,Running,11,167.610,7,indy500,0,125


In [59]:
# Get a list of cities from race_result_df and convert it to a data frame
race_numbers = race_result_df.groupby('City')['City'].count()
race_numbers_df = pd.DataFrame(race_numbers)

# Define each race's race number
race_numbers_df.at['stpetersburg', 'Race Number'] = 1
race_numbers_df.at['longbeach', 'Race Number'] = 2
race_numbers_df.at['barber', 'Race Number'] = 3
race_numbers_df.at['indyrc', 'Race Number'] = 4
race_numbers_df.at['indy500', 'Race Number'] = 5
race_numbers_df.at['detroit', 'Race Number'] = 6
race_numbers_df.at['roadamerica', 'Race Number'] = 7
race_numbers_df.at['lagunaseca', 'Race Number'] = 8
race_numbers_df.at['midohio', 'Race Number'] = 9
race_numbers_df.at['iowa1', 'Race Number'] = 10
race_numbers_df.at['iowa2', 'Race Number'] = 11
race_numbers_df.at['toronto', 'Race Number'] = 12
race_numbers_df.at['gateway', 'Race Number'] = 13
race_numbers_df.at['portland', 'Race Number'] = 14
race_numbers_df.at['milwaukee1', 'Race Number'] = 15
race_numbers_df.at['milwaukee2', 'Race Number'] = 16
race_numbers_df.at['nashville', 'Race Number'] = 17

#Clean the data frame
race_numbers_df['Race Number'] = race_numbers_df['Race Number'].astype('int64')
race_numbers_df = race_numbers_df.sort_values(by='Race Number')
race_numbers_df.rename(columns={'City': 'City2'}, inplace=True)
race_numbers_df.drop(columns=['City2'], inplace=True)
race_numbers_df = race_numbers_df.reset_index()

# Merge the data frames, thereby giving the race_result_df dataframe a race number 
race_result_df = pd.merge(race_result_df, race_numbers_df, on='City', how='inner')
race_result_df


,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Points (F1 Scoring),Points (IMSA Scoring),Race Number
0,1,Scott Dixon,9,5,100,02:06:07.9684,100,Running,53,78.251,2,detroit,25,376,6
1,2,Marcus Ericsson,28,9,100,02:06:08.8251,100,Running,40,78.242,2,detroit,18,342,6
2,3,Marcus Armstrong,11,19,100,02:06:12.8813,100,Running,35,78.200,4,detroit,15,312,6
3,4,Kyle Kirkwood,27,6,100,02:06:14.0933,100,Running,33,78.188,2,detroit,12,305,6
4,5,Alexander Rossi,7,16,100,02:06:16.9216,100,Running,30,78.158,4,detroit,10,275,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,23,Kyffin Simpson,4,22,86,01:50:32.2312,86,Contact,7,104.472,3,lagunaseca,0,89,8
462,24,Graham Rahal,15,19,86,01:50:33.2600,86,Contact,6,104.456,4,lagunaseca,0,82,8
463,25,Jack Harvey,18,26,82,01:44:40.7853,82,Mechanical,5,105.187,3,lagunaseca,0,65,8
464,26,Rinus VeeKay,21,20,72,01:39:43.0398,72,Mechanical,5,96.956,3,lagunaseca,0,61,8
